### Load libraries and functions

In [1]:
#Load libraries
library(Seurat)
library(stringr)
library(viridis)
library(ggplot2)
library(cowplot)
library(cluster)
library(data.table)
library(foreach)
library(doParallel)
library(proxy)
library(ComplexHeatmap)
library(circlize)
library(igraph)
library(qvalue)
library(dplyr)
library(viridis)
library(VGAM)
library(forcats)
library(grDevices)
library(graphics)
library(RColorBrewer)
library(pheatmap)
library(Cairo)
library(reshape2)
library(R.utils)
library(Rcpp)
library(parallelDist)
library(ggsci)
library(scales)
set.seed(seed = 42)
library(ADTnorm)

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject

Loading required package: viridisLite

Loading required package: iterators

Loading required package: parallel


Attaching package: ‘proxy’


The following objects are masked from ‘package:stats’:

    as.dist, dist


The following object is masked from ‘package:base’:

    as.matrix


Loading required package: grid

ComplexHeatmap version 2.11.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo

In [2]:
#load basic functions
createEmptyDf = function( nrow, ncol, colnames = c() ){
  data.frame( matrix( vector(), nrow, ncol, dimnames = list( c(), colnames ) ) )
}

tableread_fast = function(i, header=TRUE, quote="", sep=","){
  tmp = fread(i, header=header, sep=sep, quote=quote, nThread=32)
  tmp = as.data.frame(tmp)
  return(tmp)
}

### Create Seurat object and merge datasets

In [3]:
seu.list <- list()

In [4]:
##load data of First batch: TP3 ID2, ID27
#Put gene list in the target panel in the working directory ("BD_genes_human")

###Input layer
set.seed(seed = 42)

dir.input <- "data/original/SCT"
sample.names <- c("ID2", "ID27")

################################Processing layer##########################################
for(sample.name in sample.names){
    ##Load targeted gene expression data
    input_name <- str_c(dir.input, "matrix_inflection_", sep = "/") %>% str_c(sample.name, "Target.txt")
    compressed.name <- str_c(input_name, ".gz", sep = "")
    gunzip(compressed.name)
    #Create Seurat object
    matrix <- tableread_fast(input_name, header = TRUE, quote="", sep="\t")
    row.names(matrix) <- matrix$V1
    matrix <- dplyr::select(matrix, -V1)
    seu1 <- CreateSeuratObject(counts=matrix, project = "seu", min.cells = 3, min.features = 10)
    BC <- row.names(seu1@meta.data)
    gzip(input_name)

    ##Load protein expression data
    input_name <- str_replace(input_name,"Target", "Abseq") %>% str_replace("matrix_inflection", "Hashtag_top1M")
    compressed.name <- str_c(input_name, ".gz", sep = "")
    gunzip(compressed.name)
    #Extract cells in SCT seurat object
    Abseq.matrix <- tableread_fast(input_name, header = TRUE, quote="", sep="\t")
    Abseq.matrix <- dplyr::filter(Abseq.matrix, CellBC %in% BC)
    row.names(Abseq.matrix) <- Abseq.matrix$CellBC
    Abseq.matrix <- dplyr::select(Abseq.matrix, -CellBC)
    Abseq.matrix <- t(Abseq.matrix)
    gzip(input_name)

    #Certify
    all.equal(rownames(seu1@meta.data), colnames(Abseq.matrix))

    ##Combine ADT data into SCT seurat object
    adt_assay <- CreateAssayObject(counts = Abseq.matrix)
    seu1[["ADT"]] <- adt_assay
    Assays(seu1)

    seu1@meta.data$orig.ident <- str_c("TP3", sample.name, sep = "_")
    seu.list <- c(seu.list, list(seu1))
}

In [5]:
##load data of Second batch: TP3/TP8, ID11, 20, 23, 33, 37
#Put gene list in the target panel in the working directory ("BD_genes_human")

###Input layer
set.seed(seed = 42)

dir.input <- "data/original/SCT"
sample.names <- c("TP38Target1", "TP38Target2")

################################Processing layer##########################################
for(sample.name in sample.names){
    input_name <- str_c(dir.input, "matrix_inflection_demulti_", sep = "/") %>% str_c(sample.name, ".txt")
    compressed.name <- str_c(input_name, ".gz", sep = "")
    gunzip(compressed.name)
    #Create Seurat object
    matrix <- tableread_fast(input_name, header = TRUE, quote="", sep="\t")
    row.names(matrix) <- matrix$V1
    matrix <- dplyr::select(matrix, -V1)
    seu1 <- CreateSeuratObject(counts=matrix, project = "seu", min.cells = 3, min.features = 10)
    BC <- str_replace(row.names(seu1@meta.data), "not_detected", "notdetected") %>% str_split(pattern = "_", simplify = TRUE)
    BC <- BC[,3]
    seu1@assays$RNA@data@Dimnames[[2]] <- BC  
    row.names(seu1@meta.data) <- BC  
    gzip(input_name)

    ##Load protein expression data
    input_name <- str_replace(input_name,"Target", "Abseq") %>% str_replace("matrix_inflection_demulti", "Hashtag_top1M")
    compressed.name <- str_c(input_name, ".gz", sep = "")
    gunzip(compressed.name)
    #Extract cells in SCT seurat object
    Abseq.matrix <- tableread_fast(input_name, header = TRUE, quote="", sep="\t")
    Abseq.matrix <- dplyr::filter(Abseq.matrix, CellBC %in% BC)
    row.names(Abseq.matrix) <- Abseq.matrix$CellBC
    Abseq.matrix <- dplyr::select(Abseq.matrix, -CellBC)
    Abseq.matrix <- t(Abseq.matrix)
    gzip(input_name)
    
    #Certify
    all.equal(seu1@assays$RNA@data@Dimnames[[2]], colnames(Abseq.matrix))

    ##Combine ADT data into SCT seurat object
    adt_assay <- CreateAssayObject(counts = Abseq.matrix)
    seu1[["ADT"]] <- adt_assay
    Assays(seu1)

    #transfer Sampletag to Sample ID
    seu1@meta.data$orig.ident <- str_replace_all(seu1@meta.data$orig.ident, pattern = c("humanSampleTag10" = "TP8_ID35"))
    seu1@meta.data$orig.ident <- str_replace_all(seu1@meta.data$orig.ident, pattern = c("humanSampleTag1" = "TP3_ID10",
                                                                                       "humanSampleTag2" = "TP3_ID19",
                                                                                       "humanSampleTag3" = "TP3_ID22",
                                                                                       "humanSampleTag4" = "TP3_ID31",
                                                                                       "humanSampleTag5" = "TP3_ID35",
                                                                                       "humanSampleTag6" = "TP8_ID10",
                                                                                       "humanSampleTag7" = "TP8_ID19",
                                                                                       "humanSampleTag8" = "TP8_ID22",
                                                                                       "humanSampleTag9" = "TP8_ID31"))
    seu.list <- c(seu.list, list(seu1))
}


In [6]:
##Merge data

#Input layer
dir.SCT <- "result/intermediate/2_SCT"
sample.name <- "COVID_merged"

########################## Processing layer ###############################
dir.create(dir.SCT, recursive = TRUE)
seu <- merge(x=seu.list[[1]], y = c(seu.list[[2]], seu.list[[3]], seu.list[[4]]),
             add.cell.ids = c("Batch1-ID2", "Batch1-ID27", "Batch2-1", "Batch2-2"))

#Append batch name and cell barcodes
meta <- str_split(row.names(seu@meta.data), pattern = "_", simplify = TRUE)
seu@meta.data$batch <- meta[,1]
seu@meta.data$BC <- meta[,2]

name.output <- str_c(dir.SCT, sample.name, sep = "/") %>% str_c("rda", sep = ".")
save(seu, file = name.output)

Warning message in dir.create(dir.SCT, recursive = TRUE):
“'result/intermediate/3_SCT' already exists”


# Correct Batch Effect / 1st Analysis

In [7]:
###Exclude CD4+ cells and doublet

##Input layer
dir.SCT <- "result/intermediate/2_SCT"
dir.name="Seurat_plots"
sample.name <- "COVID_CD8only"
gene.list.name <- "metadata/BD_genes_human.txt"

########################## Processing layer ###############################
dir.name <- str_c(dir.SCT, dir.name, sep = "/")
dir.create(dir.name, recursive = TRUE)

##load gene list of BD target panel
BD_genes <- read.table(gene.list.name, header = TRUE)
BD_genes <- as.vector(BD_genes$Genesymbol)

##Chose BD target gene list for downstream analysis
res1 = seu@assays$RNA@counts
res1 =res1[res1@Dimnames[[1]] %in% BD_genes,]
ngenes <- length(res1@Dimnames[[1]])
seu@assays$RNA@counts <- res1
seu@assays$RNA@data <- res1

#Normalize Abseq data
DefaultAssay(seu) <- "ADT"
seu <- NormalizeData(seu, normalization.method = "CLR", margin = 2, assay = "ADT")
# Variable feature: All ADT Features
VariableFeatures(seu) <- rownames(seu[["ADT"]])
#Scaling data 
seu = ScaleData(object = seu)

##Exclude CD4+ cells and doublet
DefaultAssay(seu) <- "ADT"
seu <- subset(seu, subset = CD4 < 3 & CD8 > 1.5)
##Exclude doublets
#str(seu)
seu <- subset(seu, subset = orig.ident %in% c("TP3_ID2", "TP3_ID27", "TP8_ID19", "TP3_ID31", "TP8_ID31",
                                                  "TP8_ID10", "TP3_ID19", "TP8_ID35", "TP8_ID22", "TP3_ID10",
                                                  "TP3_ID35", "TP3_ID22"))

#Output Seurat object in initial analysis
file.name <- str_c(dir.SCT, sample.name, sep = "/") %>% str_c("rda", sep = ".")
save(seu, file=file.name)

Normalizing across cells

Centering and scaling data matrix



In [8]:
##Input layer
dir.SCT <- "result/intermediate/2_SCT"
sample.name <- "COVID_ADTnorm_RPCA.wnn"
dir.name="Seurat_plots"
resol <- 0.25 # resolution for clustering
genes_CLR <- c("CCR7", "CD11c", "CD25", "CD45RA", "CD127",
              "CD134", "CD137", "CD183", "CD196", "CD272",
              "CD278", "CD279", "CXCR6", "GITR", "HLA-DR", "Tim3")
genes_ADT <- c("CD27", "CD28", "CD56", "CD62L", "CD161")

########################## Processing layer ###############################
dir.name <- str_c(dir.SCT, dir.name, sep = "/")
dir.create(dir.name)

###Correct batch effect in mRNA using RPCA
#Batch1 dataset
seu.T <- subset(seu, subset = batch %in% c("Batch1-ID2", "Batch1-ID27"))
DefaultAssay(seu.T) <- 'RNA'
seu.T <- NormalizeData(seu.T, scale.factor=1000000)
VariableFeatures(seu.T) <- rownames(seu.T[["RNA"]])
#Batch2 dataset
seu.L <- subset(seu, subset = batch %in% c("Batch2-1", "Batch2-2"))
DefaultAssay(seu.L) <- 'RNA'
seu.L <- NormalizeData(seu.L, scale.factor=1000000)
VariableFeatures(seu.L) <- rownames(seu.L[["RNA"]])
seu.list <- list(seu.T, seu.L)

# select features that are repeatedly variable across datasets for integration run PCA on each
# dataset using these features
features <- SelectIntegrationFeatures(object.list = seu.list)
features
seu.list <- lapply(X = seu.list, FUN = function(x) {
    x <- ScaleData(x, features = features, verbose = FALSE)
    x <- RunPCA(x, features = features, verbose = FALSE)
})

###Perform integration
immune.anchors <- FindIntegrationAnchors(object.list = seu.list,
                                         anchor.features = features,
                                         reduction = "rpca",
                                         max.features = 50)
# this command creates an 'RNA.RPCA' data assay
seu <- IntegrateData(anchorset = immune.anchors,
                     new.assay.name = "RNA.RPCA")
DefaultAssay(seu) <- "RNA.RPCA"
seu <- ScaleData(seu, verbose = FALSE)
#Perform PCA
VariableFeatures(seu) <- rownames(seu[["RNA.RPCA"]])
seu = RunPCA(object = seu, npcs = 50)

#JackStraw
seu = JackStraw(object = seu, reduction = "pca", num.replicate = 100, dims = 50)
seu <- ScoreJackStraw(object = seu, reduction = "pca", dims = 1:50, score.thresh = 0.05)
file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("Jackstraw.png", sep='.')
png(file.name, width = 1250, height = 500)
JackStrawPlot(object = seu, dims = 1:50)
dev.off()
#Determine PCs used for clustering/tSNE analysis (dims.use)
#Extract PCs which fulfill the pvalue threshold
tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
tmp1 = tmp[tmp$Score>0.05,1]
dims= c(1:(min(tmp1)-1))
dims

###Perform ADTnorm

#Exclude genes unrelated with CD8 function
#Excluded: CD14, CD16, CD19, CD3, CD4, CD8, CXCR5, IgD, IgM
genes_analyze <- c(genes_CLR, genes_ADT)
counts <- GetAssayData(seu, slot="counts", assay="ADT")
#str(counts)
counts.sub <- counts[genes_analyze,]
#str(counts.sub)
adt_assay <- CreateAssayObject(counts = counts.sub)
seu[["ADT"]] <- adt_assay

##Process CLRnormalization
DefaultAssay(seu) <- "ADT"
seu <- NormalizeData(seu, normalization.method = "CLR", margin = 2, assay = "ADT")

##Process ADTnorm
#call coutn data and batch information
cell_x_adt <- as.data.frame(t(as.matrix(seu@assays$ADT@counts)))
cell_x_adt <- dplyr::select(cell_x_adt, genes_ADT)
cell_x_feature <- seu@meta.data
cell_x_feature$sample = factor(cell_x_feature$batch)
cell_x_feature$batch = factor(cell_x_feature$batch)
#Process
cell_x_adt_norm = ADTnorm(
    cell_x_adt = cell_x_adt, 
    cell_x_feature = cell_x_feature, 
    save_outpath = dir.name, 
    study_name = "ADTnorm",
    peak_type = "mode",
    marker_to_process = NULL, ## setting it to NULL by default will process all available markers in cell_x_adt.
    bimodal_marker = NULL, ## setting it to NULL will trigger ADTnorm to try different settings to find biomodal peaks for all the markers.
    brewer_palettes = "Dark2", ## colorbrewer palettes setting for the density plot
    save_intermediate_fig = TRUE
)

##Merge CLR and ADTnorm
aft.norm <- as.data.frame(t(as.matrix(seu@assays$ADT@data)))
aft.norm <- dplyr::select(aft.norm, -genes_ADT)
aft.norm <- cbind(aft.norm, cell_x_adt_norm)
seu@assays$ADT@data <- t(aft.norm)

##Perform wnn analysis
##For Abseq
DefaultAssay(seu) <- 'ADT'
#Normalizing and scaling data
VariableFeatures(seu) <- rownames(seu[["ADT"]])
seu = ScaleData(object = seu) 
#Perform PCA
seu = RunPCA(object = seu, reduction.name = 'apca')
    
#Clustering Weighted Nearest Neighbor
seu <- FindMultiModalNeighbors(seu, reduction.list = list("pca", "apca"), 
                                dims.list = list(dims, dims))
seu <- FindClusters(object = seu, graph.name = "wsnn", algorithm = 3, resolution =resol)

seu <- RunUMAP(seu, nn.name = "weighted.nn", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")

#Output Seurat object in initial analysis
file.name <- str_c(dir.SCT, sample.name, sep = "/") %>% str_c("res", resol, "rda", sep='.')
save(seu, file=file.name)

Warning message in dir.create(dir.name):
“'result/intermediate/3_SCT/Seurat_plots' already exists”


[1] "ADA"       "ADGRE1"    "ADGRG3"    "AIM2"      "ALAS2"     "ANXA5"    
  [7] "AOC3"      "APOBEC3G"  "APOE"      "AQP9"      "ARG1"      "ARL4C"    
 [13] "ATF6B"     "AURKB"     "AZU1"      "B3GAT1"    "BACH2"     "BAX"      
 [19] "BCL11B"    "BCL2"      "BCL2A1"    "BCL6"      "BIN2"      "BIRC3"    
 [25] "BLK"       "BLNK"      "BPI"       "BTG1"      "BTLA"      "C1QA"     
 [31] "CASP5"     "CBLB"      "CCL1"      "CCL17"     "CCL19"     "CCL20"    
 [37] "CCL22"     "CCL3"      "CCL4"      "CCL5"      "CCND2"     "CCR1"     
 [43] "CCR10"     "CCR2"      "CCR3"      "CCR4"      "CCR5"      "CCR7"     
 [49] "CCR8"      "CCR9"      "CD14"      "CD160"     "CD163"     "CD1B"     
 [55] "CD1C"      "CD2"       "CD200"     "CD209"     "CD22"      "CD24"     
 [61] "CD244"     "CD247"     "CD27"      "CD274"     "CD28"      "CD300A"   
 [67] "CD33"      "CD34"      "CD36"      "CD37"      "CD38"      "CD3D"     
 [73] "CD3E"      "CD3G"      "CD4"       "CD40"      "CD44"      "CD48"     
 [79] "CD5"       "CD52"      "CD6"       "CD63"      "CD69"      "CD7"      
 [85] "CD70"      "CD72"      "CD74"      "CD79A"     "CD79B"     "CD80"     
 [91] "CD86"      "CD8A"      "CD8B"      "CD9"       "CEACAM8"   "CHI3L1"   
 [97] "CHI3L2"    "CLC"       "CLEC4D"    "CLEC4E"    "CMKLR1"    "CMTM2"    
[103] "CNOT2"     "CNTNAP3"   "CR2"       "CSF2"      "CSF3"      "CST7"     
[109] "CTLA4"     "CTSD"      "CTSG"      "CTSW"      "CX3CR1"    "CXCL10"   
[115] "CXCL13"    "CXCL16"    "CXCL2"     "CXCL3"     "CXCL5"     "CXCL8"    
[121] "CXCR1"     "CXCR2"     "CXCR3"     "CXCR4"     "CXCR5"     "CXCR6"    
[127] "DOCK8"     "DPP4"      "DUSP1"     "DUSP2"     "DUSP4"     "EBF1"     
[133] "EGR1"      "EGR3"      "ELANE"     "ENTPD1"    "EOMES"     "F13A1"    
[139] "F5"        "FAS"       "FASLG"     "FCER1G"    "FCER2"     "FCGR3A"   
[145] "FCN1"      "FLT3"      "FN1"       "FOSB"      "FOSL1"     "FOXO1"    
[151] "FOXP1"     "FOXP3"     "FTH1"      "FUT4"      "FYN"       "GAB2"     
[157] "GAPDH"     "GIMAP2"    "GIMAP5"    "GNAI2"     "GNLY"      "GZMA"     
[163] "GZMB"      "GZMH"      "GZMK"      "HAVCR2"    "HLA-A"     "HLA-DMA"  
[169] "HLA-DPA1"  "HLA-DQB1"  "HLA-DRA"   "HMMR"      "ICAM1"     "ICOS"     
[175] "IER3"      "IFITM2"    "IFITM3"    "IFNG"      "IFNGR1"    "IGBP1"    
[181] "IGHD"      "IGHE"      "IGHG1"     "IGHG2"     "IGHG4"     "IGHM"     
[187] "IGKC"      "IKZF1"     "IKZF2"     "IL12A"     "IL12RB1"   "IL12RB2"  
[193] "IL13"      "IL15"      "IL15RA"    "IL18"      "IL18R1"    "IL18RAP"  
[199] "IL1B"      "IL1R2"     "IL1RL1"    "IL1RN"     "IL2"       "IL21"     
[205] "IL22"      "IL23R"     "IL2RA"     "IL2RB"     "IL32"      "IL33"     
[211] "IL3RA"     "IL4"       "IL4R"      "IL5"       "IL6"       "IL7R"     
[217] "IL9"       "IRF4"      "IRF8"      "ITGA4"     "ITGAE"     "ITGAM"    
[223] "ITGAX"     "ITGB2"     "JCHAIN"    "JUN"       "JUNB"      "KCNE3"    
[229] "KDELR1"    "KIR2DL1"   "KIT"       "KLRB1"     "KLRC1"     "KLRC3"    
[235] "KLRC4"     "KLRF1"     "KLRG1"     "KLRK1"     "LAG3"      "LAIR2"    
[241] "LAMP1"     "LAMP3"     "LAP3"      "LAT"       "LAT2"      "LCK"      
[247] "LEF1"      "LGALS1"    "LGALS3"    "LGALS9"    "LIF"       "LILRB4"   
[253] "LIPA"      "LRRC32"    "LTA"       "LTB"       "LY86"      "LYN"      
[259] "MCM2"      "MCM4"      "MGST1"     "MITF"      "MKI67"     "MME"      
[265] "MMP12"     "MMP9"      "MS4A1"     "MYC"       "MZB1"      "NAMPT"    
[271] "NCAM1"     "NCR3"      "NINJ2"     "NKG7"      "NRP1"      "NT5E"     
[277] "PASK"      "PCNA"      "PDCD1"     "PDIA4"     "PDIA6"     "PIK3AP1"  
[283] "PIK3IP1"   "PMCH"      "POU2AF1"   "PRDM1"     "PRF1"      "PSEN1"    
[289] "PTGDR2"    "PTPRC"     "PTTG2"     "QPCT"      "RGS1"      "RNASE2"   
[295] "RNASE6"    "RORA"      "RORC"      "RPN2"      "RUNX3"     "S100A10"  
[301] "S100A12"   "S100A9"    "SELL"      "SELPLG"    "SLC25A37"  "SLC7A7"   
[307] "SNCA"      "STAT1

Scaling features for provided objects

Computing within dataset neighborhoods

Finding all pairwise anchors

Projecting new data onto SVD

Projecting new data onto SVD

Finding neighborhoods

Finding anchors

	Found 4558 anchors

Merging dataset 1 into 2

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from rna.rpca_ to rnarpca_”
Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from rna.rpca_ to rnarpca_”
Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from rna.rpca_ to rnarpca_”
Warning message in PrepDR(object = object, features = features, verbose = verbose):
“The following 4 features requested have zero variance (running reduction without them): AQP9, CCL17, IL33, EPX”
PC_ 1 
Positive: 

Formal class 'Seurat' [package "SeuratObject"] with 13 slots
  ..@ assays      :List of 3
  .. ..$ RNA     :Formal class 'Assay' [package "SeuratObject"] with 8 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:3268808] 7 11 12 13 17 22 23 27 37 38 ...
  .. .. .. .. .. ..@ p       : int [1:53195] 0 75 134 207 277 319 375 431 475 557 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 368 53194
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:368] "ADA" "ADGRE1" "ADGRG3" "AIM2" ...
  .. .. .. .. .. .. ..$ : chr [1:53194] "Batch1-ID2_28" "Batch1-ID2_80" "Batch1-ID2_185" "Batch1-ID2_229" ...
  .. .. .. .. .. ..@ x       : num [1:3268808] 353 361 111 1 252 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:3268808] 7 11 12 13 17 22 23 27 37 38 ...
  .. .. .. .. .. ..@ p       : int

Warning message:
“Removed 16232 rows containing missing values (`geom_point()`).”


...

Warning message:
“Removed 1 rows containing missing values (`geom_segment()`).”


Progress:  Each dot is a curve
....
[1] "CD161"
                 [,1]     [,2]
Batch1-ID2  1.2456451 5.246698
Batch1-ID27 1.0007316 4.593460
Batch2-1    1.1639998 5.532452
Batch2-2    0.9598868 4.470966
Progress:  Each dot is a curve
....


Centering and scaling data matrix

Warning message in irlba(A = t(x = object), nv = npcs, ...):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning message in irlba(A = t(x = object), nv = npcs, ...):
“did not converge--results might be invalid!; try increasing work or maxit”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
PC_ 1 
Positive:  CD45RA, CD11c, HLA-DR, Tim3, CD56, CD278, GITR, CD137, CD62L, CD134 
Negative:  CD28, CD127, CD27, CD161, CD196, CXCR6, CD25, CCR7, CD279, CD18

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 53194
Number of edges: 1168338

Running smart local moving algorithm...
Maximum modularity in 10 random starts: 0.9273
Number of communities: 108
Elapsed time: 69 seconds


97 singletons identified. 11 final clusters.

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
09:11:28 UMAP embedding parameters a = 0.9922 b = 1.112

09:11:28 Commencing smooth kNN distance calibration using 1 thread

09:11:30 Initializing from normalized Laplacian + noise

09:11:31 Commencing optimization for 200 epochs, with 1703272 positive edges

09:11:53 Optimization finished



# Check the result of 1st Analysis

In [9]:
###Function: DimPlot for cluster distribution
##Input: seurat object
##Output: DimplotStraw Plot

DimOrigin <- function(seu, dir.name, sample.name, red.use, resol){
    p1 = DimPlot(object = seu, reduction = red.use, label = TRUE, label.size = 10, pt.size = 0.5) +
      theme(axis.title.x = element_text(size=10, family = "Arial"), 
            axis.title.y = element_text(size=10, family = "Arial"), 
            axis.text.x = element_text(size=10, colour = 1, family = "Arial"), 
            axis.text.y = element_text(size = 10, colour = 1, family = "Arial")) +
      theme(panel.border = element_rect(fill = NA, size = 1)) 

    legend1 <- cowplot::get_legend(p1)
    p1 = p1 + theme(legend.position = 'none')
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c(red.use, "reso", resol, "png", sep='.')
    save_plot(file = file.name, plot_grid(p1, legend1, ncol=2, nrow=1), device="png", 
              units="in", dpi = 600, base_width = 10, base_height = 5, limitsize=FALSE)
}

In [10]:
###Function: Find marker genes with marker heatmap
##Input: seurat object
##Output: Marker gene table and marker gene heatmap plot

MarkerHeatmap <- function(seu, dir.name, sample.name, resol){
    seu.markers = FindAllMarkers(seu, verbose = TRUE, test.use="wilcox", only.pos=TRUE, min.pct=0.1, features.use = NULL, return.thresh=0.05)

    #Create heatmap with top10 marker genes
    top10 = seu.markers %>% group_by(cluster) %>% top_n(10, avg_log2FC)
    top10 = as.data.frame(top10)
    top10  = top10 [!duplicated(top10$gene),]
    top10 = top10 %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
    top10 = as.data.frame(top10)
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("marker_res", resol, "png", sep='.')
    p <- DoHeatmap(seu, features = top10$gene, disp.min = -2.5, disp.max = 2.5, size = 8)
    ggsave(file = file.name, plot = p, device="png", units="in", dpi = 300,
           width = 20, height = 20, limitsize=FALSE)

    seu.markers$cluster = as.numeric(seu.markers$cluster)
    seu.markers = seu.markers %>% arrange(desc(avg_log2FC))  %>% arrange(cluster)
    seu.markers = as.data.frame(seu.markers)
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("ALLmarkers_minpct0.1_Adj_p0.05.txt", sep='')
    fwrite(seu.markers, file.name, row.names=F, col.names=T, sep="\t", quote=F)
    file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("ALLmarkers_minpct0.1_Adj_p0.05.rda", sep='')
    save(seu.markers, file=file.name)
}

In [11]:
##Plot clones in dimentional reduction plot (DimPlot)
#seu: Seurat object / tmp2: identifier of plotting cells

PlotClones = function(seu, tmp2, name.output, meta.data, reduction, width, height, size){
    #Exctract names for cells in plot
    tmp_raw <- as.vector(row.names(meta.data))
    #Extract colors for plot
    p2 = Seurat::DimPlot(object = seu, reduction = reduction, pt.size = 0.1)
    orig_ident_build = ggplot2::ggplot_build(p2)
    orig_ident_build = orig_ident_build$data[[1]]
    orig_ident_build =  orig_ident_build[order(orig_ident_build$group), ]
    ident.cols = unique(orig_ident_build$colour) # Get a vector of unique colors
    names(ident.cols)=c(0:(max(as.numeric(seu@active.ident)-1)))

    #Extract x- and y- axis information for DimPlot
    xmin <- min(orig_ident_build$x)-0.5
    xmax <- max(orig_ident_build$x)+0.5
    ymin <- min(orig_ident_build$y)-0.5
    ymax <- max(orig_ident_build$y)+0.5
    x_label_min <- floor(xmin/10)*10
    x_label_max <- ceiling(xmax/10)*10
    y_label_min <- floor(ymin/10)*10
    y_label_max <- ceiling(ymax/10)*10

    #Dimplot
    p1 <- DimPlot(object = seu, label = FALSE,
                    cells = tmp2,
                    reduction = reduction,
                    cols = ident.cols,
                    pt.size = size) + 
        NoLegend() +
        scale_x_continuous(limits = c(xmin,xmax), breaks= seq(x_label_min,x_label_max,10)) +
        scale_y_continuous(limits = c(ymin,ymax), breaks= seq(y_label_min,y_label_max,10)) +
        theme(axis.title = element_blank(), 
              axis.text = element_blank(),
             axis.line = element_line(size = 0.2))
    
      save_plot(file = name.output, plot_grid(p1, ncol=1, nrow=1), device="tiff", 
                units="in", dpi = 300, base_width = width, base_height = height, limitsize=FALSE)
}


In [12]:
###Plot bar graph for cluster distribution
Cluster_bar <- function(seu, red.use, data, dir.name, file.name, width, height){
    #Extract colors for plot
    p2 = Seurat::DimPlot(object = seu, reduction = red.use, pt.size = 0.1)
    orig_ident_build = ggplot2::ggplot_build(p2)
    orig_ident_build = orig_ident_build$data[[1]]
    orig_ident_build =  orig_ident_build[order(orig_ident_build$group), ]
    ident.cols = unique(orig_ident_build$colour) # Get a vector of unique colors
    names(ident.cols)=c(0:(max(as.numeric(seu@active.ident)-1)))

    #Plot bar graph
    ppi <- 600
     g <- ggplot(data.table, aes(x = Treat, y = Number, fill = clust)) +
        scale_fill_manual(values=ident.cols) +
        geom_bar(stat = "identity", position = "fill") +
        labs(y="Prop. of cluster") +
        scale_y_continuous(labels = percent) +
        theme_classic() + 
        theme(plot.title = element_blank(), 
              panel.background = element_rect(fill = "transparent",color = NA),
              plot.background = element_rect(fill = "transparent",color = NA),
                axis.title.x = element_blank(), 
                axis.title.y = element_text(size=8, colour = 1, family = "Arial"), 
                axis.text.x = element_text(size = 6, colour = 1, family = "Arial", angle = 45), 
                axis.text.y = element_text(size = 6, colour = 1, family = "Arial")) +
        guides(fill = "none") 
    save_plot(file = file.name, plot(g), device="tiff", 
              units="in", dpi = 600, base_width = width, base_height = height, limitsize=FALSE)
}

In [13]:
###Function: Create Scatter and Violin plot for selected genes
##Input: seurat object, gene list
##Output: Scatter plot, Violin plot

ScatterViolin <- function(seu, dir.name, sample.name, red.use, tmps, tmp_names){
    for (i in 1:length(tmp_names)){
        #Call gene list
        tmp <- tmps[[i]]
        tmp_name <- tmp_names[i]
  
        #Scatter Plot
        file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("Scatter", tmp_name, "png", sep='.')
        png(file.name, width = 1536, height = 240)
        p <- FeaturePlot(seu, features = tmps, ncol = 6, order = TRUE,
                        reduction = red.use, dims=c(1,2), cols = c("grey", "red"), pt.size = 0.2)
        plot(p)
        dev.off()
        
        #Violin Plot
        file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("Violin", tmp_name, "png", sep='.')
        png(file.name, width = 4608, height = 720)
        p <- VlnPlot(seu, features = tmps, ncol = 6, pt.size = 0.1)
        plot(p)
        dev.off()
    }
}

In [14]:
##Create Dim Plot
##Check lineage marker gene expression

###Input layer
dir.SCT <- "result/intermediate/2_SCT"
sample.name <- "COVID.ADTnorm_RPCA.wnn"
dir.name <- "Seurat_plots"
red.use <- "wnn.umap"
resol <- 0.25 # resolution for clustering
tmp_rna = c("TRBC2", "TRDC", "CD14", "MS4A1", "KLRB1", "ZBTB16")
tmp_adt = c("CD62L", "CD45RA", "CD56", "CD196", "CD161", "CXCR6")

########################## Processing layer #############################
dir.name <- str_c(dir.SCT, dir.name, sep = "/")
dir.create(dir.name)

#DimPlot with sample origin
DimOrigin(seu, dir.name, sample.name, red.use, resol)

###RNA
DefaultAssay(seu) <- 'RNA.RPCA'
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, str_c(sample.name, "RNA"), resol)
##Check lineage marker gene expression
ScatterViolin(seu, dir.name, str_c(sample.name, "RNA"), red.use, tmp_rna, "lineage")

###ADT
DefaultAssay(seu) <- 'ADT'
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, str_c(sample.name, "ADT"), resol)
##Check lineage marker gene expression
ScatterViolin(seu, dir.name, str_c(sample.name, "ADT"), red.use, tmp_adt, "lineage")


Warning message in dir.create(dir.name):
“'result/intermediate/3_SCT/Seurat_plots' already exists”
Warning message:
“The `size` argument of `element_rect()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Arial' n

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warnin

# 2nd analysis with removing gd T cells

In [15]:
##Input layer
seu <- subset(seu, idents = c(0, 1, 2, 3, 4, 5, 6, 8, 9, 10))#7: gd T cells
str(seu)

dir.SCT <- "result/intermediate/2_SCT"
sample.name <- "COVID_ADTnorm_RPCA.wnn"
dir.name <- "Seurat_plots_2nd"
red.use <- "wnn.umap"
resol <- 0.25 # resolution for clustering

########################## Processing layer #############################
dir.name <- str_c(dir.SCT, dir.name, sep = "/")
dir.create(dir.name, recursive = TRUE)

###RNA
DefaultAssay(seu) <- "RNA.RPCA"
#Scale data and Perform PCA
VariableFeatures(seu) <- rownames(seu[["RNA.RPCA"]])
seu <- ScaleData(seu, verbose = FALSE)
seu = RunPCA(object = seu, npcs = 50)
#JackStraw
seu = JackStraw(object = seu, reduction = "pca", num.replicate = 100, dims = 50)
seu <- ScoreJackStraw(object = seu, reduction = "pca", dims = 1:50, score.thresh = 0.05)
file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("Jackstraw.png", sep='.')
png(file.name, width = 1250, height = 500)
JackStrawPlot(object = seu, dims = 1:50)
dev.off()
#Determine PCs used for clustering/tSNE analysis (dims.use)
#Extract PCs which fulfill the pvalue threshold
tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
tmp1 = tmp[tmp$Score>0.05,1]
dims= c(1:(min(tmp1)-1))
dims

###ADT
DefaultAssay(seu) <- 'ADT'
#Scale data and Perform PCA
VariableFeatures(seu) <- rownames(seu[["ADT"]])
seu = ScaleData(object = seu) 
seu = RunPCA(object = seu, reduction.name = 'apca')
    
###Clustering Weighted Nearest Neighbor
seu <- FindMultiModalNeighbors(seu, reduction.list = list("pca", "apca"), 
                                dims.list = list(dims, c(1:10)))

seu <- RunUMAP(seu, nn.name = "weighted.nn", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")

###Output Seurat object in initial analysis
file.name <- str_c(dir.SCT, sample.name, sep = "/") %>% str_c("_2nd.rda", sep='')
save(seu, file=file.name)

Formal class 'Seurat' [package "SeuratObject"] with 13 slots
  ..@ assays      :List of 3
  .. ..$ RNA     :Formal class 'Assay' [package "SeuratObject"] with 8 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:3185490] 7 11 12 13 17 22 23 27 37 38 ...
  .. .. .. .. .. ..@ p       : int [1:51854] 0 75 134 207 277 319 375 431 475 557 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 368 51853
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:368] "ADA" "ADGRE1" "ADGRG3" "AIM2" ...
  .. .. .. .. .. .. ..$ : chr [1:51853] "Batch1-ID2_28" "Batch1-ID2_80" "Batch1-ID2_185" "Batch1-ID2_229" ...
  .. .. .. .. .. ..@ x       : num [1:3185490] 353 361 111 1 252 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:3185490] 7 11 12 13 17 22 23 27 37 38 ...
  .. .. .. .. .. ..@ p       : int

Warning message in PrepDR(object = object, features = features, verbose = verbose):
“The following 4 features requested have zero variance (running reduction without them): AQP9, CCL17, IL33, EPX”
PC_ 1 
Positive:  GZMH, GZMB, FCGR3A, GNLY, NKG7, LGALS1, CX3CR1, PRF1, CST7, ZNF683 
	   ITGAM, ITGB2, CCL5, IFNG, GZMA, APOBEC3G, B3GAT1, CD63, RUNX3, CD300A 
	   KLRC3, LAIR2, CTSW, KIR2DL1, CD8B, CTSD, KLRK1, CD8A, CD3G, CCL4 
Negative:  IL7R, LTB, GZMK, TCF7, KLRB1, TNFRSF25, CD27, MYC, ZBTB16, RORC 
	   TRAT1, CCR7, DPP4, JUNB, DUSP2, PIK3IP1, HLA-A, PASK, IL18RAP, CCR2 
	   CXCR4, DUSP1, CD69, CXCR6, LEF1, NCR3, NT5E, CD28, BIRC3, BTG1 
PC_ 2 
Positive:  GNLY, CCL5, NKG7, IGLC3, GZMB, ZNF683, S100A12, CD1C, ALAS2, F13A1 
	   LIF, SNCA, CD80, IL5, TLR2, IGHG2, GZMH, MMP12, CXCL8, ADGRG3 
	   CTSG, FCER2, CD33, CXCL2, CXCR2, CD14, CLEC4D, ZBED2, CNTNAP3, MME 
Negative:  CD27, DUSP1, CD69, LCK, GAPDH, CD74, JUNB, RGS1, FOSB, DUSP2 
	   BTG1, CD3E, CD48, HLA-DRA, CD6, CXCR3, GIMAP2, FYN, C

png 
  2

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15

Centering and scaling data matrix

Warning message in irlba(A = t(x = object), nv = npcs, ...):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning message in irlba(A = t(x = object), nv = npcs, ...):
“did not converge--results might be invalid!; try increasing work or maxit”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
PC_ 1 
Positive:  CD45RA, CD11c, HLA-DR, Tim3, CD56, CD278, GITR, CD137, CD62L, CD272 
Negative:  CD28, CD127, CD27, CD161, CD196, CXCR6, CD25, CCR7, CD279, CD18

In [16]:
###Check clustering resolution / Merker genes for each cluster
dir.SCT <- "result/intermediate/2_SCT"
sample.name <- "COVID.ADTnorm_RPCA.wnn"
dir.name <- "Seurat_plots_2nd"
red.use <- "wnn.umap"
tmp_rna = c("TRBC2", "TRDC", "CD14", "MS4A1", "KLRB1", "ZBTB16")
tmp_adt = c("CD62L", "CD45RA", "CD56", "CD196", "CD161", "CXCR6")
resol = 0.28

########################## Processing layer #############################
dir.name <- str_c(dir.SCT, dir.name, sep = "/")
dir.create(dir.name, recursive = TRUE)

#Find Clusters
seu <- FindClusters(object = seu, graph.name = "wsnn", algorithm = 3, resolution =resol)

#DimPlot with sample origin
DimOrigin(seu, dir.name, sample.name, red.use, resol)
    
###RNA
DefaultAssay(seu) <- 'RNA.RPCA'
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, str_c(sample.name, "RNA"), resol)
##Check lineage marker gene expression
ScatterViolin(seu, dir.name, str_c(sample.name, "RNA"), red.use, tmp_rna, "lineage")

###ADT
DefaultAssay(seu) <- 'ADT'
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, str_c(sample.name, "ADT"), resol)
##Check lineage marker gene expression
ScatterViolin(seu, dir.name, str_c(sample.name, "ADT"), red.use, tmp_adt, "lineage")

###Output Seurat object in initial analysis
file.name <- str_c(dir.SCT, sample.name, sep = "/") %>% str_c("2nd.res", resol, "rda", sep='.')
save(seu, file=file.name)

Warning message in dir.create(dir.name, recursive = TRUE):
“'result/intermediate/3_SCT/Seurat_plots_2nd' already exists”


Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 51853
Number of edges: 1297731

Running smart local moving algorithm...
Maximum modularity in 10 random starts: 0.9168
Number of communities: 73
Elapsed time: 84 seconds


62 singletons identified. 11 final clusters.

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial'

Calculating cluster 9

Calculating cluster 10

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10



# 3rd analysis with removing MAIT cells

In [17]:
##Input layer 
seu <- subset(seu, idents = c(0, 1, 3, 4, 5, 6, 8, 10)) #2,9: MAIT cells

dir.SCT <- "result/intermediate/2_SCT"
sample.name <- "COVID_ADTnorm_RPCA.wnn"
dir.name <- "Seurat_plots_3rd"
red.use <- "wnn.umap"

########################## Processing layer #############################
dir.name <- str_c(dir.SCT, dir.name, sep = "/")
dir.create(dir.name, recursive = TRUE)

###RNA
DefaultAssay(seu) <- "RNA.RPCA"
#Scale data and Perform PCA
VariableFeatures(seu) <- rownames(seu[["RNA.RPCA"]])
seu <- ScaleData(seu, verbose = FALSE)
seu = RunPCA(object = seu, npcs = 50)
#JackStraw
seu = JackStraw(object = seu, reduction = "pca", num.replicate = 100, dims = 50)
seu <- ScoreJackStraw(object = seu, reduction = "pca", dims = 1:50, score.thresh = 0.05)
file.name <- str_c(dir.name, sample.name, sep='/') %>% str_c("Jackstraw.png", sep='.')
png(file.name, width = 1250, height = 500)
JackStrawPlot(object = seu, dims = 1:50)
dev.off()
#Determine PCs used for clustering/tSNE analysis (dims.use)
#Extract PCs which fulfill the pvalue threshold
tmp = as.data.frame(seu@reductions$pca@jackstraw@overall.p.values)
tmp1 = tmp[tmp$Score>0.05,1]
dims= c(1:(min(tmp1)-1))
dims

###ADT
DefaultAssay(seu) <- 'ADT'
#Scale data and Perform PCA
VariableFeatures(seu) <- rownames(seu[["ADT"]])
seu = ScaleData(object = seu) 
seu = RunPCA(object = seu, reduction.name = 'apca')
    
###Clustering Weighted Nearest Neighbor
seu <- FindMultiModalNeighbors(seu, reduction.list = list("pca", "apca"), 
                                dims.list = list(dims, c(1:10)))

seu <- RunUMAP(seu, nn.name = "weighted.nn", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")

###Output Seurat object in initial analysis
file.name <- str_c(dir.SCT, sample.name, sep = "/") %>% str_c("_3rd.rda", sep='')
save(seu, file=file.name)

Warning message in PrepDR(object = object, features = features, verbose = verbose):
“The following 5 features requested have zero variance (running reduction without them): AQP9, CCL17, IL33, EPX, SPP1”
PC_ 1 
Positive:  GZMH, NKG7, GNLY, GZMB, CST7, FCGR3A, PRF1, GZMA, CCL5, CX3CR1 
	   LGALS1, ZNF683, CTSW, ITGAM, ITGB2, CD63, CD300A, B3GAT1, RUNX3, APOBEC3G 
	   KIR2DL1, KLRC3, KLRF1, CCL4, LAIR2, CTSD, IFNG, CD244, CD247, ITGAX 
Negative:  TCF7, IL7R, LTB, CD27, CCR7, GZMK, PASK, LEF1, JUNB, PIK3IP1 
	   TNFRSF25, TRAT1, MYC, DUSP2, FOSB, CD69, SELL, YBX3, DUSP1, CXCR4 
	   HLA-A, CXCR3, NT5E, CD28, RGS1, CD79A, BTG1, JUN, BIRC3, CD5 
PC_ 2 
Positive:  IGLC3, ALAS2, S100A12, CD1C, F13A1, GNLY, SNCA, ADGRG3, CNTNAP3, IGHG2 
	   TLR2, NT5E, IL5, IL9, CD80, CXCL8, TREM1, LIF, CCL5, CD14 
	   CXCL2, CTSG, IL4, TNFRSF4, CLEC4D, MMP12, APOE, JCHAIN, MME, CCL20 
Negative:  GAPDH, CD74, HLA-DRA, LCK, CD3G, ITGB2, ADA, GNAI2, TYMS, APOBEC3G 
	   BIN2, IL2RB, CD8A, TOP2A, ANXA5, PTPRC, CD3D,

png 
  2

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16

Centering and scaling data matrix

Warning message in irlba(A = t(x = object), nv = npcs, ...):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning message in irlba(A = t(x = object), nv = npcs, ...):
“did not converge--results might be invalid!; try increasing work or maxit”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
Warning message:
“Requested number is larger than the number of available items (21). Setting to 21.”
PC_ 1 
Positive:  CD27, CD28, CD127, CCR7, CD279, CD25, CD183, CD272, CXCR6, CD161 
Negative:  CD45RA, CD56, CD11c, Tim3, GITR, HLA-DR, CD137, CD62L, CD196, CD27

In [18]:
###Check clustering resolution / Merker genes for each cluster
sample.name <- "COVID"
dir.SCT <- "result/intermediate/2_SCT"
dir.name <- "Seurat_plots_3rd"
red.use <- "wnn.umap"
tmp_rna = c("TRBC2", "TRDC", "CD14", "MS4A1", "KLRB1", "ZBTB16")
tmp_adt = c("CD62L", "CD45RA", "CD56", "CD196", "CD161", "CXCR6")
resol = 0.2

########################## Processing layer #############################
dir.name <- str_c(dir.SCT, dir.name, sep = "/")
dir.create(dir.name, recursive = TRUE)

#Find Clusters
seu <- FindClusters(object = seu, graph.name = "wsnn", algorithm = 3, resolution =resol)

#DimPlot with sample origin
DimOrigin(seu, dir.name, sample.name, red.use, resol)
    
###RNA
DefaultAssay(seu) <- 'RNA.RPCA'
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, str_c(sample.name, "RNA"), resol)
##Check lineage marker gene expression
ScatterViolin(seu, dir.name, str_c(sample.name, "RNA"), red.use, tmp_rna, "lineage")

###ADT
DefaultAssay(seu) <- 'ADT'
#Marker gene extraction and create marker gene heatmap
MarkerHeatmap(seu, dir.name, str_c(sample.name, "ADT"), resol)
##Check lineage marker gene expression
ScatterViolin(seu, dir.name, str_c(sample.name, "ADT"), red.use, tmp_adt, "lineage")

###Output Seurat object in initial analysis
file.name <- str_c(dir.SCT, sample.name, sep = "/") %>% str_c("3rd.res", resol, "rda", sep='.')
save(seu, file=file.name)

Warning message in dir.create(dir.name, recursive = TRUE):
“'result/intermediate/3_SCT/Seurat_plots_3rd' already exists”


Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 41852
Number of edges: 1053523

Running smart local moving algorithm...
Maximum modularity in 10 random starts: 0.9174
Number of communities: 73
Elapsed time: 57 seconds


64 singletons identified. 9 final clusters.

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' 

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Warning message in FindMarkers.default(ob

# Removing minor clusters

In [19]:
seu <- subset(seu, idents = c(0, 1, 2, 3, 4, 5))

###Visualize clusters
sample.name <- "COVID_ADTnorm_RPCA.wnn"
dir.SCT <- "result/intermediate/2_SCT"
dir.name <- "Seurat_plots_3rd_womin"
red.use <- "wnn.umap"
tmp_rna = c("TRBC2", "TRDC", "CD14", "MS4A1", "KLRB1", "ZBTB16")
tmp_adt = c("CD62L", "CD45RA", "CD56", "CD196", "CD161", "CXCR6")
resol = 0.2

########################## Processing layer #############################
dir.name <- str_c(dir.SCT, dir.name, sep = "/")
dir.create(dir.name, recursive = TRUE)

#DimPlot with sample origin
DimOrigin(seu, dir.name, sample.name, red.use, resol)
    
#Marker gene extraction and create marker gene heatmap (RNA)
DefaultAssay(seu) <- 'RNA.RPCA'
MarkerHeatmap(seu, dir.name, str_c(sample.name, "RNA"), resol)
DefaultAssay(seu) <- 'ADT'
MarkerHeatmap(seu, dir.name, str_c(sample.name, "ADT"), resol)

##Check lineage marker gene expression
DefaultAssay(seu) <- 'RNA.RPCA'
ScatterViolin(seu, dir.name, str_c(sample.name, "RNA"), red.use, tmp_rna, "lineage")
DefaultAssay(seu) <- 'ADT'
ScatterViolin(seu, dir.name, str_c(sample.name, "ADT"), red.use, tmp_adt, "lineage")    

#Output Seurat object in initial analysis
file.name <- str_c(dir.SCT, sample.name, sep = "/") %>% str_c("3rd_womin.res", resol, "rda", sep='.')
save(seu, file=file.name)

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warnin

Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Warning message in grid.Call(C_textBounds, as.graphicsAnnot(x$label), x$x, x$y, :
“font family 'Arial' not found in PostScript font database”
Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5



# Integration of scTCRseq results into SCT Seurat object

In [20]:
###Define function
#Create histogram was showing the total number of TCR reads detected in each cell (top left) 
#and the percentage of TCR reads that were the most common among them (bottom left).
#Scatter plot representing these two metrices of cells were also generated
Histogram <- function(d, file.name, sample.name, dir.output){

  d_count_total <- as.numeric(d$count.total)
  
  #Count histogram
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c(sample.name, "histogram.count.tiff", sep = '.') 
  tiff(image.file, width=1.2*ppi, height=0.8*ppi, res=ppi)
  p <- ggplot(NULL, aes(x=d_count_total)) +
    geom_histogram(binwidth=0.1) +
    theme_bw(base_size = 6) +
    labs(x = "Read counts of TCR") +
    theme(
      axis.title.y=element_blank(),
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) +
    scale_x_continuous(trans=scales::log2_trans(),
                   breaks=scales::trans_breaks("log2",function(x) 2^x),
                   labels=scales::trans_format("log2",scales::math_format(2^.x)))
  print(p)
  dev.off()

  #Frequency histogram  
  d_freq <- as.numeric(d$freq)
  med_f <- 0.3
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c(sample.name, "histogram.freq.tiff", sep = '.')
  tiff(image.file, width=1.2*ppi, height=0.8*ppi, res=ppi)
  p <- ggplot(NULL, aes(x=d_freq)) +
    geom_histogram(binwidth=0.05) +
    theme_bw(base_size = 6) +
    labs(x = "Proportion of the largest TCR read") +
    theme(
      axis.title.y=element_blank(),
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) 
  print(p)
  dev.off()

  df <- data.frame(d_count_total, d_freq)
  
  #Scatter plot    
  total <- nrow(subset(df, df$d_freq >= 0))
  lb <- paste(round(100*nrow( subset(df, df$d_freq < 0.6 & df$d_count_total < 2^5 )) / total, digits = 1), "%", sep="" ) 
  rb <- paste(round(100*nrow( subset(df, df$d_freq >= 0.6 & df$d_count_total < 2^5 )) / total, digits = 1), "%", sep="" ) 
  lt <- paste(round(100*nrow( subset(df, df$d_freq < 0.6 & df$d_count_total >= 2^5 )) / total, digits = 1), "%", sep="" ) 
  rt <- paste(round(100*nrow( subset(df, df$d_freq >= 0.6 & df$d_count_total >= 2^5 )) / total, digits = 1), "%", sep="" ) 
  
  ppi <- 300
  image.file <- str_c(dir.output, file.name, sep = "/") %>% str_c(sample.name, "Scatter.tiff", sep = '.')                     
  tiff(image.file, width=1.2*ppi, height=1.2*ppi, res=ppi)
  p <- ggplot(d, aes(x=d_freq, y=d_count_total)) +  
    stat_bin2d(bins=60) +
    scale_fill_gradient(low="lightblue", high="red") +
    theme_bw(base_size = 6) +
    geom_vline(aes(xintercept = 0.6), size=0.25, colour="black") +
    geom_hline(aes(yintercept = 2^5), size=0.25, colour="black") +
    labs(x = "Proportion of the largest TCR read", y = "Read counts of TCR") +
    theme(
      axis.text.x = element_text(family="Arial"),
      axis.text.y = element_text(family="Arial"),
      axis.title=element_text(size=4)) +
    guides(fill=FALSE) +
    scale_y_continuous(trans=scales::log2_trans(),
                   breaks=scales::trans_breaks("log2",function(x) 2^x),
                   labels=scales::trans_format("log2",scales::math_format(2^.x)))
    p <- p + #xlim(0,1) +
    annotate("text", x=-Inf, y=0, hjust=-0.1, vjust=-0.4, label=lb,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=Inf, y=0, hjust=1.1, vjust=-0.4, label=rb,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=-Inf, y=Inf, hjust=-0.1, vjust=1.3, label=lt,
             family="Arial",colour="black",size=1.5) +
    annotate("text", x=Inf, y=Inf, hjust=1.1, vjust=1.3, label=rt,
             family="Arial",colour="black",size=1.5) 
  print(p)
  dev.off()  
}

In [21]:
###Define function
#Create TCRa/TCRb combine table for cell barcodes
CombineTable <- function(dir.input, sample.name, dir.output, count_th, freq_th, clonotype, cores){
    combined.tables <- data.frame()
    
    ###Define function
    tableread_fast = function(i, header=TRUE, quote="", sep=","){
      tmp = fread(i, header=header, sep=sep, quote=quote, nThread=32)
      tmp = as.data.frame(tmp)
      return(tmp)
    }

    #Convert MiXCR output into VDJtools format
    Convert <- function(file.name, name.TRA){
        name.input <- str_c(name.TRA, file.name, sep = "/")
        data <- tableread_fast(name.input, header = TRUE, sep = '\t')

        #Extract TCR information from mixcr output
        count.total<-data$cloneCount / data$cloneFraction
        freq<-data$cloneFraction
        cdr3nt<-data$nSeqImputedCDR3
        cdr3aa<-data$aaSeqImputedCDR3
        v<-str_sub(data$bestVHit, end=-4)
        d<-str_sub(data$bestDHit, end=-4)
        j<-str_sub(data$bestJHit, end=-4)
        data3 <- rbind(count.total,freq,cdr3nt,cdr3aa,v,d,j)
        data3 <- as.data.frame(t(data3))
        names(data3) <- c("count.total","freq","cdr3nt","cdr3aa","v","d","j")

        #Extract largest clone in cell barcode and append cell barcode information
        name_out <- str_split(file.name, "_")
        CB <- name_out[[1]][[4]]
        CB_out <- str_split(CB, "\\.")
        CB <- CB_out[[1]][[1]]
        d_out <- data3[1,]
        d_out$CB <- CB

        return(d_out)
    }
    
    #Convert MiXCR output into VDJtools format
    name.TRA <- str_c(sample.name, "TRAC1_mixcr", sep = "_")
    files  <- list.files(name.TRA, pattern=".txt")
    cl <- makeCluster(cores)
    registerDoParallel(cl)
    TRA.table <- invisible(foreach(file.name = files,
            .combine = rbind, .packages=c("ggplot2", "extrafont", "stringr", "dplyr", "data.table")) %dopar% {Convert(file.name, name.TRA)})
    stopCluster(cl)
    unlink(name.TRA, recursive=TRUE)

    name.TRB <- str_c(sample.name, "TRBC1_mixcr", sep = "_")
    files  <- list.files(name.TRB, pattern=".txt")
    cl <- makeCluster(cores)
    registerDoParallel(cl)
    TRB.table <- invisible(foreach(file.name = files,
            .combine = rbind, .packages=c("ggplot2", "extrafont", "stringr", "dplyr", "data.table")) %dopar% {Convert(file.name, name.TRB)})
    stopCluster(cl)
    unlink(name.TRB, recursive=TRUE)

    #Output histogram and scatter plot for summarizing scTCR status
    Histogram(TRA.table, "TRA", sample.name, dir.output)
    Histogram(TRB.table, "TRB", sample.name, dir.output)

    #thresholding
    TRA.table$count.total <- as.numeric(TRA.table$count.total)
    TRA.table$freq <- as.numeric(TRA.table$freq)
    TRB.table$count.total <- as.numeric(TRB.table$count.total)
    TRB.table$freq <- as.numeric(TRB.table$freq)
    TCRa_th <- subset(TRA.table, count.total >= 2^(count_th) & freq >= freq_th)
    TCRb_th <- subset(TRB.table, count.total >= 2^(count_th) & freq >= freq_th)

    #Paring TCRa and TCRb by cell barcode
    names(TCRa_th) <- c("count.total.A","freq.A","cdr3nt.A","cdr3aa.A","v.A","d.A","j.A", "CB")
    names(TCRb_th) <- c("count.total.B","freq.B","cdr3nt.B","cdr3aa.B","v.B","d.B","j.B", "CB")
    combined <- merge(TCRa_th, TCRb_th, all=T, by ="CB")
    combined$CB <- str_c(sample.name, combined$CB, sep = "_")
    #Exclude cells in which neither TCRa nor TCRb sequence were detected
    combined <- subset(combined, combined$cdr3nt.A != "UD" | combined$cdr3nt.B != "UD")

    #Generate clone id for further analysis
    #Definition of clones can be changed between "ABnt", "ABaa", "Bnt", and "Baa"
    if(clonotype=="nt"){
        combined$clone.id.TCRa <- str_c(combined$cdr3nt.A, combined$v.A, combined$j.A, sep="_")
        combined$clone.id.TCRb <- str_c(combined$cdr3nt.B, combined$v.B, combined$j.B, sep="_")
    }
    if(clonotype=="aa"){
        combined$clone.id.TCRa <- str_c(combined$cdr3aa.A, combined$v.A, combined$j.A, sep="_")
        combined$clone.id.TCRb <- str_c(combined$cdr3aa.B, combined$v.B, combined$j.B, sep="_")
    }
    combined$clone.id.TCRab <- str_c(combined$clone.id.TCRa, combined$clone.id.TCRb, sep = "_")
    name.output <- str_c(dir.output, sample.name, sep = "/") %>% str_c(clonotype, "table", "count_th", count_th, "freq_th", freq_th, clonotype, "csv", sep = ".")
    write.csv(combined, name.output, row.names = FALSE)
    
    combined.tables <- rbind(combined.tables, combined)
    
    return(combined.tables)
}

In [22]:
#Extract TCR sequence for each cell barcode
#iteration process for dataset (batches)
set.seed(seed = 42)
sample.names <- c("ID02TCR", "ID28TCR", "TP38TCR1", "TP38TCR2")
dir.input <- "data/original/SCT"
cores <- 12
dir.SCT <- "result/intermediate/2_SCT"
dir.output <- "scTCR_processing"
#Threshold for valid cell barcode with single TCR sequence
count_th <- 5 #read count threshold for all TCR sequences per cell barcode
freq_th <- 0.6 #proportion threshold for largest TCR
clonotype <- "nt" #Definition of clones. nt: nucleotide sequence / aa: amino acid sequnece 

################################ Processing layer #################################################
dir.output <- str_c(dir.SCT, dir.output, sep = "/")
dir.create(dir.output)

#Unzip files
files <- list.files(dir.input, ".tar.bz2")
for(file in files){
    name.input <- str_c(dir.input, file, sep = "/")
    bunzip2(name.input, remove=FALSE)
    name.input <- str_remove(name.input, pattern = ".bz2")
    untar(name.input)
    file.remove(name.input)
}

combined.tables <- data.frame()

for(sample.name in sample.names){
    ##Create TCRa/TCRb combine table for cell barcodes
    combined.table <- CombineTable(dir.input, sample.name, dir.output, count_th, freq_th, clonotype, cores)
    combined.tables <- rbind(combined.tables, combined.table)
}

###Merge TCR info into meta.data
combined.tables$CB <- str_replace(combined.tables$CB, "ID02TCR", "Batch1-ID2")
combined.tables$CB <- str_replace(combined.tables$CB, "ID28TCR", "Batch1-ID27")
combined.tables$CB <- str_replace(combined.tables$CB, "TP38TCR1", "Batch2-1")
combined.tables$CB <- str_replace(combined.tables$CB, "TP38TCR2", "Batch2-2")

#Extract meta.data and cell BC information
meta.data <- seu@meta.data
meta.data$CB <- row.names(meta.data)

#Merge to Seurat object of SCT data
meta.data <- merge(combined.tables, meta.data, all.y = T, by ="CB")
clone.ids <- c("clone.id.TCRa", "clone.id.TCRb", "clone.id.TCRab")
for(clone.id in clone.ids){
    clone.id.list <- dplyr::select(meta.data, clone.id)
    row.names(clone.id.list)=as.character(meta.data$CB)
    seu <- AddMetaData(object = seu, metadata = clone.id.list, col.name = clone.id)
}

#Assign TP and ID information
ID_TP <- str_split(seu@meta.data$orig.ident, pattern = "_", simplify = TRUE)
seu@meta.data$TP <- ID_TP[,1]
seu@meta.data$ID <- ID_TP[,2]

#Output Seurat object in initial analysis
file.name <- str_c(dir.SCT, "COVID_ADTnorm_RPCA.wnn_3rd.res.0.2.wo-min.scTCR.rda", sep = "/")
save(seu, file=file.name)

Warning message:
“Removed 2538 rows containing non-finite values (`stat_bin()`).”
Warning message:
“Removed 2538 rows containing non-finite values (`stat_bin()`).”
Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Warning message:
“The `<scale>` argument of `guides()` cannot be `FALSE`. Use "none" instead as
of ggplot2 3.3.4.”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Removed 2538 rows containing non-finite values (`stat_bin2d()`).”
Warning message:
“Removed 2646 rows containing non-finite values (`stat_bin()`).”
Warning message:
“Removed 2646 rows containing non-finite values (`stat_bin()`).”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Transformation introduced infinite values in continuous y-axis”
Warning message:
“Removed 

# Introducing Signature Scores from published SCT data

In [23]:
###Define function
##Plotting Signature scores for clusters
modify_vlnplot_score <- function(obj, 
                          feature, 
                          pt.size = 0, 
                          plot.margin = unit(c(-0.75, 0, -0.75, 0), "cm"),
                          ...) {
  
  p<- VlnPlot(obj, features = feature, pt.size = pt.size)  + 
    xlab("") + ylab(feature) + ggtitle("") + 
    ggplot2::geom_boxplot(outlier.size = 0, size = 0.2) + 
    theme(legend.position = "none",
          plot.title= element_blank(),
          axis.title.x = element_blank(),
          axis.text.x =  element_text(family = "Arial", size = 8, angle = 0, colour = "black", face = "plain"), 
          axis.ticks.x = element_blank(), 
          axis.title.y = element_blank(), 
          axis.text.y = element_text(family = "Arial", size = 6, angle = 0, colour = "black", face = "plain"), 
          plot.margin = plot.margin ) 
  return(p)
}

In [24]:
###Generate Gene list for Signature Scoring

dir.input <- "metadata"
name.input <- "Score.Gene.list.txt"
name.reference <- "BD_genes_human.txt"
dir.output <- "result/intermediate/2_SCT/GeneScoreSource"

######################## Processing layer #########################
dir.create(dir.output)

d <- read.table(str_c(dir.input, name.input, sep = "/"), header = TRUE)
d$clust <- str_c(d$Source, d$Name, sep = ".")
ref <- read.csv(str_c(dir.input, name.reference, sep = "/"), header = TRUE)
name.clusts <- unique(d$clust)
for(name.clust in name.clusts){
    d_sub <- dplyr::filter(d, clust == name.clust)
    g.ext <- unique(intersect(d_sub$gene, ref$Genesymbol))
    d_out <- as.data.frame(g.ext)
    names(d_out) <- "Genes"
    name.output <- str_c(dir.output, "Sig", sep = "/") %>% str_c(name.clust, "txt", sep = ".")
    write.table(d_out, name.output, row.names = FALSE)
}

In [25]:
# Introduce Gene Scores using AddModuleScore function

#Put signature gene set into "Signatures" directory
dir.input <- "result/intermediate/2_SCT/GeneScoreSource"
dir.SCT <- "result/intermediate/2_SCT"

########################## Processing layer #############################
#load Signature data
files  <- list.files(dir.input, pattern="Sig.")

for(i in files){
    i <- str_c(dir.input, i, sep = "/")
    gene_list <- read.table(i, header = TRUE)
    gene_list <- list(as.vector(gene_list$Genes))
    sig_name <- str_remove(i, dir.input)
    sig_name <- str_remove(sig_name, "/Sig.")
    sig_name <- str_remove(sig_name, ".txt")
    seu <- AddModuleScore(object = seu, 
                            ctrl=5,
                            features = gene_list,
                            assay="RNA.RPCA",
                            name=sig_name,
                         )
}

#Output Seurat object
file.name <- str_c(dir.SCT, "COVID_ADTnorm_RPCA.wnn_3rd.res.0.2.wo-min.scTCR.Sig.rda", sep = "/")
save(seu, file=file.name)

Warning message:
“The following features are not present in the object: KIAA0101, not searching for symbol synonyms”
Warning message:
“The following features are not present in the object: IFNA1, not searching for symbol synonyms”
Warning message:
“The following features are not present in the object: FAM65B, not searching for symbol synonyms”
Warning message:
“The following features are not present in the object: FYB, not searching for symbol synonyms”
Warning message:
“The following features are not present in the object: FYB, not searching for symbol synonyms”


# Integration of Bulk TCRseq result

In [26]:
##Make clone table by assembling clones from scTCR results 
CloneSummary <- function(seu, dir.name, sample.name, clonotype){
    #seu: Seurat object for analyze
    #dir.name: directory name for output
    dir.create(dir.name)

    #load metadata
    meta.data <- as.data.frame(seu@meta.data)
    #extract cells only in the assigned sample
    meta.data <- dplyr::filter(meta.data, orig.ident == sample.name)
    
    #Define clone by TCRa / TCRb/ TCRa&b
    if(clonotype == "TCRa"){
       meta.data$clone.id <- meta.data$clone.id.TCRa 
    }
    if(clonotype == "TCRb"){
       meta.data$clone.id <- meta.data$clone.id.TCRb 
    }
    if(clonotype == "TCRab"){
       meta.data$clone.id <- meta.data$clone.id.TCRab 
    } 

    #Summarize cluster distribution of each clone
    tmp_out = table(meta.data$seurat_clusters, meta.data$clone.id)
    tmp_out2 <- as.data.frame(tmp_out, row.names = NULL,
                  responseName = "Freq", stringsAsFactors = TRUE,
                  sep = "", base = list(LETTERS))
    names(tmp_out2) <- c("Clust", "names", "Freq")
    tmp_out2 <- dcast(tmp_out2, names ~ Clust)

    #Summarize the count and frequency of clones 
    tmp = meta.data %>% group_by(`clone.id`) %>%
      dplyr::summarise(count = n()) %>%
      dplyr::arrange(desc(count))
    tmp = as.data.frame(tmp)
    tmp <- tmp[!is.na(tmp$clone.id), ] #NAである細胞を除く
    tmp[,3]=tmp[,2]/sum(tmp[,2])
    colnames(tmp)=c("ntSeq_TRA_TRB_freq", "CloneCount", "CloneFreq")
    tmp <- tmp[order(tmp$ntSeq_TRA_TRB_freq),]
    temp_count <- as.vector(tmp$CloneCount)
    temp_freq <- as.vector(tmp$CloneFreq)
    #Combine to the cluster distribution
    tmp_out3 <- cbind(temp_count, temp_freq, tmp_out2)

    #Assign ranks to clones
    tmp_out3 <- tmp_out3[order(tmp_out3$temp_freq, decreasing=T),]
    rank <-  1:nrow(tmp_out3)
    tmp_out3 <- cbind(rank, tmp_out3)
    tmp_out3 <- data.frame(tmp_out3)
    tmp_out3$rank <- paste("Top",tmp_out3$rank, sep="")
    
    #Output
    name.out <- str_c(dir.name, sample.name, sep = "/") %>% str_c(clonotype, "clone_within_cluster.txt", sep = ".")
    write.table(tmp_out3, name.out, row.names=F, col.names=T, sep="\t", quote=F) 
}

In [27]:
##Make clone table by assembling clones from scTCR results 

dir.name <- "result/intermediate/2_SCT/scTCR_analysis"

##################### Processing layer #############################################################
dir.create(dir.name)

sample.list <- unique(seu@meta.data$orig.ident)

for(sample.name in sample.list){
    #Make clone table by assembling clones from scTCR results 
    CloneSummary(seu, dir.name, sample.name, "TCRb")
} 


Warning message in dir.create(dir.name):
“'result/intermediate/3_SCT/scTCR_analysis' already exists”
Using Freq as value column: use value.var to override.

Warning message in dir.create(dir.name):
“'result/intermediate/3_SCT/scTCR_analysis' already exists”
Using Freq as value column: use value.var to override.

Warning message in dir.create(dir.name):
“'result/intermediate/3_SCT/scTCR_analysis' already exists”
Using Freq as value column: use value.var to override.

Warning message in dir.create(dir.name):
“'result/intermediate/3_SCT/scTCR_analysis' already exists”
Using Freq as value column: use value.var to override.

Warning message in dir.create(dir.name):
“'result/intermediate/3_SCT/scTCR_analysis' already exists”
Using Freq as value column: use value.var to override.

Warning message in dir.create(dir.name):
“'result/intermediate/3_SCT/scTCR_analysis' already exists”
Using Freq as value column: use value.var to override.

Warning message in dir.create(dir.name):
“'result/intermed

# Integration of Bulk TCRseq result

In [4]:
###Merge SCT metadata with WGCNA pattern / AIM positivity

#Input layer
dir.input.scTCR <- "result/intermediate/2_SCT/scTCR_analysis"
dir.input.bulk <- "result/intermediate/1_beta-binomial/JoinTP_DifAbund"
dir.output <- "result/intermediate/2_SCT/scTCR_analysis"

sample.names <- c("002", "027", "010", "019", "022", "031", "035")
sample.scts <- c("ID2.TCR", "ID27.TCR", "ID10.TCR", "ID19.TCR", "ID22.TCR", "ID31.TCR", "ID35.TCR")

##################### Processing layer #############################################################
for(i in 1:length(sample.names)){
    #load data
    sct.names <- list.files(dir.input.scTCR, sample.scts[i])
    for(sct.name in sct.names){
        sct <- read.table(str_c(dir.input.scTCR, sct.name, sep = "/"), header = TRUE)
        bulk.name <- str_c(dir.input.bulk, "NaraCOVID_CD8", sep = "/") %>%
            str_c(sample.names[i], "TCR.DifAbund.csv", sep = "_")
        bulk <- read.csv(bulk.name, header = TRUE)
        
        #Merge bulk and sct dataset
        d_output <- merge(sct, bulk, by.x = "names", by.y = "ntvj", all.x = T)
        
        #Output
        name.out <- str_c(dir.output, sct.name, sep = "/") %>%
                        str_replace("clone_within_cluster.txt", "clone_within_cluster.bulkinfo.csv")
        write.csv(d_output, name.out, row.names = FALSE)
    }
}